In [ ]:
import pandas as pd

In [ ]:
# MODIFICATION : Ajoute le dictionnaire de types ici
types_fixes = {
    "Code postal de l'établissement": str,
    "Code du département de l'établissement": str,
    "SIREN": str,
    "Code commune de l'établissement": str
}

df = pd.read_csv("data.csv", dtype=types_fixes, sep=";", low_memory=False)
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

---

#### Vérification des SIREN, NIC pour valider les drop

In [ ]:
# On s'assure que les colonnes sont comparables (en string sans le .0)
siret_str = df["SIRET"].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(14)
siege_str = df["SIRET du siège de l'unité légale"].astype(str).str.replace(r'\.0$', '', regex=True).str.zfill(14)

# 1. Vérification de la concordance SIRET ↔ SIRET du siège

siret_ok = (siret_str == siege_str)

# 2. Vérification de la cohérence SIREN ↔ SIRET
siren_nunique = df["SIREN"].nunique()
siret_nunique = df["SIRET"].nunique()

# Résumé
print(f"Nombre de SIREN uniques : {siren_nunique}")
print(f"Nombre de SIRET uniques : {siret_nunique}")
print("Cohérence SIREN / SIRET (SIREN <= SIRET) :", siren_nunique <= siret_nunique)

# Optionnel : combien d'établissements sont des sièges ?
print(f"Nombre de sièges identifiés : {siret_ok.sum()}")

In [ ]:
# On crée un masque booléen sécurisé

is_siege = (
    df["SIRET"].astype(str).str.replace(r'\.0$', '', regex=True) == 
    df["SIRET du siège de l'unité légale"].astype(str).str.replace(r'\.0$', '', regex=True)
)

sieges_par_siren = (
    df[is_siege]
    .groupby("SIREN")
    .size()
)

print("Tous les SIREN ont un seul siège :", (sieges_par_siren == 1).all())


---

#### Drop des colonnes inutiles selon l'études des sociétés fermées

In [ ]:
# Liste des colonnes à supprimer
colonnes_a_drop = [
    
    "Libellé cedex de l'établissement",
    "Numero de voie de l'établissement 2",
    "Distribution spéciale de l'établissement",
    "Libellé du pays de l'établissement étranger.1",
    "Prénom usuel de la personne physique",
    "Prénom de la personne physique 3",
    "Indice de répétition de l'établissement 2",
    "Complément d'adresse de l'établissement 2",
    "Code cedex de l'établissement",
    "Code pays de l'établissement",
    "Libellé cedex de l'établissement 2",
    "Code cedex de l'établissement 2",
    "Code de la commune de l'établissement 2",
    "Distribution spéciale de l'établissement 2",
    "Libellé de la commune de l'établissement étranger 2",
    "Libellé de la commune de l'établissement 2",
    "Code postal de l'établissement 2",
    "Libellé de la voie de l'établissement 2",
    "Type de voie de l'établissement 2",
    "Pseudonyme de la personne physique",
    "Prénom de la personne physique 1",
    "Civilité de la personne physique",
    "Prénom de la personne physique 4",
    "Caractère employeur de l'unité légale",
    "Première ligne de l'adressage",
    "Nom de la personne physique",
    "Nom d'usage de la personne physique",
    "Prénom de la personne physique 2",
    "Unité légale purgée",
    "Libellé de la commune de l'établissement à l'étranger",
    "Code du pays de l'établissement étranger",
    "Identifiant association de l'unité légale",
    "Libellé du pays de l'établissement étranger",
    "Dénomination usuelle de l'unité légale 3",
    "Enseigne de l'établissement 3",
    "Dénomination usuelle de l'unité légale 2",
    "Enseigne de l'établissement 2",
    "Société à mission unité légale",
    "Indice de répétition de l'établissement",
    "Dénomination usuelle de l'unité légale 1",
    "Sigle de l'unité légale",
    "Enseigne de l'établissement 1",
    "Complément d'adresse de l'établissement",
    "Année de la tranche d'effectif de l'établissement",
    "Dénomination usuelle de l'établissement",
    "Activité principale de l'établissement",
    "Année de la tranche de l'effectif de l'unité légale",
    "Etablissement siège",
    "Statut de diffusion de l'établissement",
    "Statut de diffusion de l'unité légale",
    "Tranche de l'effectif de l'établissement",
    "Tranche de l'effectif de l'établissement triable",
    "Numéro de voie de l'établissement",
    "Type de voie de l'établissement",
    "Libellé de la voie de l'établissement",
    "NIC",
    "SIRET",
    "NIC du siège de l'unité légale",
    "Activité principale de l'établissement.1",

    "Commune de l'établissement",
    "Caractère employeur de l'établissement",
    "Date du dernier traitement de l'unité légale",
    "Nombre de périodes de l'unité légale",
    "Etat administratif de l'établissement",

    "Code EPCI de l'établissement",
    "Libellé de l'EPCI de l'établissement",
    "Département de l'établissement",
    "Région de l'établissement",
    "Section de l'établissement",
    "Sous-section de l'établissement",
    "Division de l'établissement",
    "Groupe de l'établissement",
    "Adresse de l'établissement",
    "SIRET du siège de l'unité légale",
    "Date de fermeture de l'établissement",   


    "Tranche de l'effectif de l'unité légale triable", 
    "Catégorie de l'entreprise", 
    "Année de la catégorie de l'entreprise", 
    "Classe de l'établissement",
    "Section de l'unité légale",
    "Sous-section de l'unité légale",
    "Division de l'unité légale",
    "Groupe de l'unité légale",
    "Classe de l'unité légale",
    "Nombre de periodes de l'établissement",
    "Nomenclature de l'activité principale de l'unité légale",
    "Nomenclature principale de l'établissement",
    "Code de l'arrondissement de l'établissement",
    
]

# Supprimer les colonnes
df = df.drop(columns=colonnes_a_drop)

# Vérification de la nouvelle taille du dataset
print(f"Dimensions après nettoyage : {df.shape}")

In [ ]:
df.head()

In [ ]:
df.columns.to_list()

---

#### Filtrage des SAS et SARL

In [ ]:
# 🔹 Étape 1 : filtrer uniquement SAS et SARL exacte
sas_sarl_values = [
    "SAS, société par actions simplifiée",
    "Société à responsabilité limitée (sans autre indication)"
]

df_filtered = df[df["Nature juridique de l'unité légale"].isin(sas_sarl_values)]
# 🔹 Étape 2 : vérification rapide

print("Formes juridiques retenues :")
print(df_filtered["Nature juridique de l'unité légale"].value_counts())

In [ ]:
df_filtered.shape

---

#### Examen des valeurs pour décisions à prendre

In [ ]:
df_filtered.head()

In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_filtered.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

In [ ]:
df_filtered['Economie sociale et solidaire unité légale'].value_counts()

##### Remplacement des Nan dans l'ESS par des N

In [ ]:
df_filtered['Economie sociale et solidaire unité légale'] = (
    df_filtered['Economie sociale et solidaire unité légale']
    .fillna("N")
)


In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_filtered.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

In [ ]:
df_filtered['Date de fermeture de l\'unité légale'].value_counts()

In [ ]:
df_filtered = df_filtered[df_filtered["Date de fermeture de l'unité légale"].isna()]


In [ ]:
# Incrémation d'un Nan pour la Date de fermeture : les sociétés sont ouvertes
df_filtered["Date de fermeture de l'unité légale"].value_counts(dropna=False)


---

#### Remplacement de la date de fermeture par le 31 décembre 2025 en prévision du calcul de l'âge des sociétés.

In [ ]:
df_filtered["Date de fermeture de l'unité légale"] = "2025-12-31"


In [ ]:
df_filtered.head()

#### Nan restants à traiter dans le second notebook

In [ ]:
# Calculer le pourcentage de NaN par colonne
nan_percent = df_filtered.isna().mean() * 100

# Sélectionner seulement les colonnes avec au moins un NaN
nan_columns = nan_percent[nan_percent > 0].sort_values(ascending=False)

# Afficher
print(nan_columns)

In [ ]:
df_filtered.shape

---

In [ ]:
df_filtered.dtypes

In [ ]:
# 1. On règle le cas du Code Région (84.0 -> "84")
df_filtered["Code de la région de l'établissement"] = (
    df_filtered["Code de la région de l'établissement"]
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
)

# 2. Conversion en datetime (pour que le format Parquet les reconnaisse)

cols_dates = [
    "Date de création de l'établissement", 
    "Date de création de l'unité légale", 
    "Date de fermeture de l'unité légale"
]

for col in cols_dates:
    df_filtered[col] = pd.to_datetime(df_filtered[col], errors='coerce')

df_filtered.dtypes

In [ ]:
df_filtered.head()

##### Export Parquet

In [ ]:
# # 3. Exportation finale en Parquet
# df_filtered.to_parquet("sas_sarl_ouvertes.parquet", index=False)